In [4]:
import os
import pickle
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import numpy as np

print('Evaluate customer probability of loan default\n\n')
mappings = {
    'person_home_ownership': {'RENT': 0, 'OWN': 1, 'MORTGAGE': 2, 'OTHER': 3},
    'loan_intent': {'DEBTCONSOLIDATION': 0, 'EDUCATION': 1, 'HOMEIMPROVEMENT': 2, 'MEDICAL': 3, 'PERSONAL': 4, 'VENTURE': 5},
    'loan_grade': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6},
    'loan_status' : {'Default': 0, 'NonDefault': 1},
    'cb_person_default_on_file': {'N': 0, 'Y': 1},
}
columns = ['person_age', 'person_income', 'person_emp_length', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length', 'person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']

def standardize_data(df, columns_to_standardize):
    scaler = StandardScaler()
    df[columns_to_standardize] = scaler.fit_transform(df[columns_to_standardize])
    return df

display(HTML('''
    <style>
    .widget-label { min-width: 30ex !important; }
    </style>
'''))

# Directory insisde models
subdirs = os.listdir('models/')
subdir_selector = widgets.Dropdown(
    options = subdirs,
    description='Select predictors:',
    disabled = False,
)
display(subdir_selector)

# Show a widget for each indip. variable
input_widgets = {}
for column in columns:
    if column not in mappings:
        input_widgets[column] = widgets.FloatText(
            description=column,
            disabled=False,
        )
    else:
        input_widgets[column] = widgets.Dropdown(
        options=mappings.get(column, {}).keys(),
        description=column,
        disabled=False,
    )
    display(input_widgets[column])

print('\n')
# Button to get the prediction
predict_button = widgets.Button(
    description='Evaluate',
    disabled=True,
    button_style='',
    tooltip='Click to evaluate',
    icon='check',
)
display(predict_button)

# Output space
output = widgets.Output()
display(output)

def on_subdir_change(change):
    with output:
        predict_button.disabled = False
        output.clear_output()
        
    selected_predictors = list(eval(change['new']))    
    for column, widget in input_widgets.items():
        widget.disabled = column not in selected_predictors    
    
    models_dir = os.path.join('models', subdir_selector.value)
    model_files = os.listdir(models_dir)
    model_selector.options = model_files

# Register callback function
subdir_selector.observe(on_subdir_change, names='value')

# When click on predict
def on_predict_click(b):
    with output: 
        output.clear_output()
        maps = {
            'person_home_ownership': {'RENT': 0, 'OWN': 1, 'MORTGAGE': 2, 'OTHER': 3},
            'loan_intent': {'DEBTCONSOLIDATION': 0, 'EDUCATION': 1, 'HOMEIMPROVEMENT': 2, 'MEDICAL': 3, 'PERSONAL': 4, 'VENTURE': 5},
            'loan_grade': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6},
            'loan_status' : {'Default': 0, 'NonDefault': 1},
            'cb_person_default_on_file': {'N': 0, 'Y': 1},
        }
        
        # read selected model
        selected_predictors = list(eval(subdir_selector.value))
        
        # get user data
        user_data = [maps[column][input_widgets[column].value] if column in maps else input_widgets[column].value for column in selected_predictors]        
        df = pd.DataFrame(data=[user_data], columns=selected_predictors)

        # predict for each model
        predictions = []
        for file in os.listdir(os.path.join('models', subdir_selector.value)):
            if file != 'Logit.pkl':
                model = pickle.load(open(os.path.join('models', subdir_selector.value, file), 'rb'))
                prediction = model.predict_proba(df)[:, 1]
                predictions.append(prediction[0])
        
        # calculate average
        average_prediction = sum(predictions) / len(predictions)
        if average_prediction > 0.5:
            print(f"\nAverage Probability of loan default is {round(average_prediction * 100, 3)}% --> No Loan")
        else:
            print(f"\nAverage Probability of loan default is {round(average_prediction * 100, 3)}% --> Yes Loan")        

predict_button.on_click(on_predict_click)

Evaluate customer probability of loan default




Dropdown(description='Select predictors:', options=("('loan_grade', 'loan_intent', 'person_income')", "('loan_…

FloatText(value=0.0, description='person_age')

FloatText(value=0.0, description='person_income')

FloatText(value=0.0, description='person_emp_length')

...

Dropdown(description='loan_intent', options=('DEBTCONSOLIDATION', 'EDUCATION', 'HOMEIMPROVEMENT', 'MEDICAL', '…

Dropdown(description='loan_grade', options=('A', 'B', 'C', 'D', 'E', 'F', 'G'), value='A')

Dropdown(description='cb_person_default_on_file', options=('N', 'Y'), value='N')

Button(description='Forecast', disabled=True, icon='check', style=ButtonStyle(), tooltip='Click to forecast')

Output()